# OPEN AI

In [2]:
import gym
import numpy as np 

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
import matplotlib.pyplot as plt

from rl.agents.dqn import DQNAgent
from rl.agents.ddpg import DDPGAgent
from rl.policy import BoltzmannGumbelQPolicy , LinearAnnealedPolicy , EpsGreedyQPolicy
from rl.memory import SequentialMemory

Using TensorFlow backend.


In [3]:
env = gym.make('MsPacman-v0')
nb_actions = env.action_space.n
nb_actions

9

In [4]:
## Env.
env = gym.make("Pong-v0")
# env = gym.make("SpaceInvaders-v0")
obsertation = env.reset()
nb_actions = env.action_space.n
nb_actions


6

In [ ]:
##Model

In [5]:
# Next, we build a neural network model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(3, activation= 'tanh')) # layer 1: 3 cells with tanh activation function 
model.add(Dense(nb_actions))
model.add(Dense(6, activation= 'sigmoid')) #layer 2 : 6 cells with sigmoid activation function 
model.add(Dense(nb_actions))
model.add(Dense(6, activation= 'sigmoid')) #layer 3 : 6 cells with sigmoid activation function  
model.add(Dense(nb_actions))
model.add(Dense(6, activation= 'sigmoid')) #layer 4 : 6 cells with sigmoid activation function  
model.add(Dense(nb_actions))
model.add(Dense(3, activation= 'tanh')) #layer 5 : 3 cells with tanh activation function  
model.add(Dense(nb_actions))
model.add(Dense(3, activation= 'sigmoid')) #layer 6 : 6 cells with sigmoid activation function  
model.add(Dense(nb_actions))
model.add(Activation('softmax')) # one layer of 1 unit with sigmoid activation function
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 100800)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 302403    
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 24        
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_5 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_6 (Dense)              (None, 6)                 42        
__________

In [7]:
model = Sequential()
model.add(Flatten(input_shape=(1,)+env.observation_space.shape))
#RGB가 들어오니 최소 3개는 있는게 좋고 그안에서 댄스의 크기를 변화하는것이 좋음
model.add(Dense(3, activation='tanh'))
#model.add(Dense(3, activation='sigmoid'))
model.add(Dense(4, activation='tanh'))
#마지막은 선택가능한 마지막 변수로 지정
model.add(Dense(nb_actions))

model.add(Activation('softmax'))

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 100800)            0         
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 302403    
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 16        
_________________________________________________________________
dense_17 (Dense)             (None, 6)                 30        
_________________________________________________________________
activation_2 (Activation)    (None, 6)                 0         
Total params: 302,449
Trainable params: 302,449
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#Policy

In [9]:
#아헨테를 바꿀수 있어. 나중에 찾아보렴
memory = SequentialMemory(limit=100000,  window_length=1)
Policy = BoltzmannGumbelQPolicy()
#
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=50,
              target_model_update=1e-2, policy=Policy)

In [10]:
dqn.compile(Adam(lr=1e-3), metrics=['acc','mae']) #optimizador
dqn.fit(env, nb_steps = 100000, visualize=True)

Training for 100000 steps ...
Interval 1 (0 steps performed)


ReraisedException: Error occured while running `from pyglet.gl import *`
The original exception was:

ImportError: Library "GL" not found.

HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'. If you're running on a server, you may need a virtual frame buffer; something like this should work: 'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'

In [ ]:
## Visualize the training during 5000000 steps 
## Each episode is a game in wich Pacman has two lifes 
### When nb_steps is not long enough , a warning might appear 
### "Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!"

dqn.fit(env, nb_steps=100000, visualize=False, verbose=2)

Training for 100000 steps ...
  1293/100000: episode: 1, duration: 88.427s, episode steps: 1293, steps per second: 15, episode reward: -19.000, mean reward: -0.015 [-1.000, 1.000], mean action: 2.459 [0.000, 5.000], mean observation: 98.031 [0.000, 236.000], loss: 0.008547, acc: 0.204056, mean_absolute_error: 0.146434, mean_q: 0.192961
  2644/100000: episode: 2, duration: 103.222s, episode steps: 1351, steps per second: 13, episode reward: -20.000, mean reward: -0.015 [-1.000, 1.000], mean action: 2.546 [0.000, 5.000], mean observation: 97.986 [0.000, 236.000], loss: 0.009081, acc: 0.183429, mean_absolute_error: 0.143350, mean_q: 0.178466
  4176/100000: episode: 3, duration: 108.582s, episode steps: 1532, steps per second: 14, episode reward: -20.000, mean reward: -0.013 [-1.000, 1.000], mean action: 2.514 [0.000, 5.000], mean observation: 97.957 [0.000, 236.000], loss: 0.008319, acc: 0.183747, mean_absolute_error: 0.142895, mean_q: 0.176771
  5337/100000: episode: 4, duration: 86.162s

In [ ]:
callback